## Colab 환경에서 환경설정
- 앞으로의 남은 실습은 Colab에서 실행한다는 가정하에 실습을 진행합니다.

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.2.4/spark-3.2.4-bin-hadoop3.2.tgz
!tar xf spark-3.2.4-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
# findspark를 사용해 Spark 환경 초기화
import findspark

# findspark는 Python 환경에서 PySpark를 사용할 수 있도록 Spark 경로를 자동으로 설정해주는 라이브러리입니다.
# findspark.init()은 환경 변수를 설정해 PySpark가 올바르게 작동하도록 합니다.
# init() 메서드에 경로를 지정하지 않으면, 기본적으로 SPARK_HOME 환경 변수를 자동으로 검색합니다.
findspark.init("/content/spark-3.2.4-bin-hadoop3.2")

- data 디렉토리에 있는 파일을 drive의 /content 경로에 올려주세요.

## 라이브러리 및 spark 시작 및 처리를 위한 객체 생성

In [4]:
# PySpark 라이브러리를 임포트합니다. PySpark는 대규모 데이터를 병렬로 처리하고 분석하는 데 사용됩니다.
import pyspark

# DataFrame 및 SQL 작업을 수행하기 위해 SparkSession을 임포트합니다.
# SparkSession은 PySpark에서 DataFrame 및 SQL 작업을 위한 진입점 역할을 합니다.
from pyspark.sql import SparkSession

# Spark 설정을 구성하고 SparkContext를 생성하기 위해 필요한 모듈을 임포트합니다.
# SparkConf는 Spark 애플리케이션의 설정을 정의하는 데 사용되고,
# SparkContext는 RDD API를 사용하여 데이터 처리를 수행하기 위한 기본 객체입니다.
from pyspark import SparkConf, SparkContext

# 새로운 SparkSession을 생성합니다. 'prediction'은 애플리케이션 이름으로,
# Spark UI 및 로그에 표시됩니다. SparkSession은 DataFrame과 SQL 작업을 수행하는 데 사용됩니다.
spark = SparkSession.builder.appName('prediction').getOrCreate()

# SparkContext 객체를 가져옵니다. SparkContext는 RDD API 작업을 수행할 때 사용되는 기본 객체입니다.
# RDD(Resilient Distributed Dataset)는 PySpark의 저수준 API로, 데이터를 분산 처리하기 위해 사용됩니다.
sc = spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/31 12:41:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/31 12:42:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/31 12:42:00 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/10/31 12:42:00 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/10/31 12:42:00 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


## 1. 데이터 파싱 및 결측치 제거
- `parseLine` 함수: 각 행을 쉼표로 분할해 승객 ID, 생존 여부, 나이, 티켓 요금을 추출하며, 결측치는 None으로 처리합니다.
- RDD 생성: `sc.textFile`을 사용해 CSV 파일을 불러와 RDD를 만들고, 헤더 행을 제거해 본문 데이터만 남깁니다.
- 결측치 제거: `filter`를 통해 나이와 티켓 요금이 None이 아닌 행만 남겨 유효한 데이터를 확보합니다.
- 데이터 개수 확인: `count` 메서드로 정제된 데이터의 개수를 출력해 데이터 정제 결과를 확인합니다.

In [6]:
# 각 행을 파싱하는 함수 정의
def parseLine(line):
    fields = line.split(',')
    passengerID = fields[0]  # 승객 ID
    survived = int(fields[1])  # 생존 여부: 0(사망), 1(생존)
    
    # 나이와 티켓 요금을 처리할 때, 비어 있는 경우를 확인합니다.
    try:
        age = float(fields[6]) if fields[6] else None  # 비어 있으면 None으로 처리
        fare = float(fields[10]) if fields[10] else None  # 비어 있으면 None으로 처리
    except ValueError:
        age, fare = None, None  # 예외가 발생하면 None으로 처리

    return (passengerID, survived, age, fare)


In [ ]:
# CSV 파일을 RDD로 로드
data_rdd = sc.textFile("train.csv")

# 첫 번째 행(헤더) 제거
header = data_rdd.first()  # 첫 번째 행을 가져옵니다.
data_rdd = data_rdd.filter(lambda row: row != header)  # 헤더와 일치하지 않는 행만 남깁니다.

# 데이터 파싱 및 RDD 생성
parsedLines = data_rdd.map(parseLine)

# 결측치가 있는 행 제거 (age 또는 fare가 None인 경우 필터링)
parsedLines = parsedLines.filter(lambda x: x[2] is not None and x[3] is not None)

# 데이터 개수 출력
data_count = parsedLines.count()
print(f"결측치를 제거한 후의 데이터 개수: {data_count}")

# 결과 확인 (예시)
parsedLines.take(5)

결측치를 제거한 후의 데이터 개수: 714


[('1', 0, 22.0, 7.25),
 ('2', 1, 38.0, 71.2833),
 ('3', 1, 26.0, 7.925),
 ('4', 1, 35.0, 53.1),
 ('5', 0, 35.0, 8.05)]

## 2. 룰 기반 생존 예측
- `predict_survival` 함수는 나이와 티켓 요금을 기준으로 승객의 생존 여부를 예측합니다.
- **나이 기반 룰**: 5세 이하인 경우 생존 가능성이 높다고 판단하고, 60세 이상인 경우 생존 가능성이 낮다고 설정합니다.
- **티켓 요금 기반 룰**: 티켓 요금이 50달러 이상이면 생존 가능성이 높다고 보고, 20달러 이하일 경우 생존 가능성이 낮다고 가정합니다.
- **결합 룰**: 30세 이하이면서 티켓 요금이 50달러 이상인 경우 생존 가능성이 매우 높다고 판단합니다. 반대로, 30세 초과이며 티켓 요금이 20달러 이하인 경우 생존 가능성이 낮습니다.
- **기본값**: 위의 조건을 만족하지 않는 경우 기본적으로 생존 가능성이 낮다고 설정하며, 나이 또는 티켓 요금이 결측치인 경우에도 생존 가능성이 낮다고 예측합니다.
- `parsedLines` RDD에 `predict_survival` 함수를 적용하여 승객 ID, 실제 생존 여부, 예측된 생존 여부를 반환합니다.


In [ ]:
# 룰 기반 예측 함수 정의
def predict_survival(age, fare):
    if age is not None and fare is not None:
        # 나이 기반 룰
        if age <= 5:
            return 1  # 생존 가능성 높음
        elif age >= 60:
            return 0  # 생존 가능성 낮음
        
        # 티켓 요금 기반 룰
        if fare >= 50:
            return 1  # 생존 가능성 높음
        elif fare <= 20:
            return 0  # 생존 가능성 낮음
        
        # 결합 룰
        if age <= 30 and fare >= 50:
            return 1  # 생존 가능성 매우 높음
        elif age > 30 and fare <= 20:
            return 0  # 생존 가능성 낮음
        
        # 기본값
        return 0  # 생존 가능성 낮음
    else:
        return 0  # 결측치가 있는 경우 생존 가능성 낮음

# 룰 기반 예측 적용 및 정확도 계산
# TODO: predictions는 (passengerID, 실제 생존 여부, 예측된 생존 여부) 형식의 튜플로 구성된 RDD입니다.
# 이 RDD를 사용하여 예측 모델의 정확도를 평가할 수 있습니다.
# map과 lambda를 이용해서 predictions를 완성해주세요.
predictions = ___________________________________________________

# predictions의 값을 확인합니다.
print(predictions.take(5))

[('1', 0, 0), ('2', 1, 1), ('3', 1, 0), ('4', 1, 1), ('5', 0, 0)]


## 3. 정확도 계산
- **correct_predictions**: `predictions` RDD에서 실제 생존 여부(`x[1]`)와 예측된 생존 여부(`x[2]`)가 일치하는 경우를 필터링하고, 해당 개수를 `count`로 구합니다. 즉, 올바른 예측의 총 개수를 계산합니다.
- **total_predictions**: `predictions` RDD의 전체 개수를 `count`로 구하여 총 예측된 데이터 개수를 확인합니다.
- **accuracy**: 정확도를 계산하기 위해 올바른 예측의 개수를 전체 예측 개수로 나누어줍니다. 결과는 0과 1 사이의 값으로 표현되며, 이를 100을 곱해 퍼센트 형태로 변환합니다.
- **결과 출력**: 총 예측 개수, 정확한 예측 개수, 그리고 예측 정확도를 소수점 두 자리까지 출력하여 모델의 성능을 확인할 수 있도록 합니다.

In [ ]:
# 정확도 계산
# TODO: correct_predictions를 채우기 위한 함수를 완성하세요.
# filter를 계산하고 난 결과를 기반으로 map 함수와 reduce 함수를 사용하여 정확한 예측의 총 개수를 계산하세요.
correct_predictions = predictions.filter(lambda x: x[1] == x[2])._______________________________

# total_predictions 계산
total_predictions = predictions.map(lambda x: 1).reduce(lambda a, b: a + b)
accuracy = correct_predictions / total_predictions

# 결과 출력
print(f"총 예측 개수: {total_predictions}")
print(f"정확한 예측 개수: {correct_predictions}")
print(f"예측 정확도: {accuracy * 100:.2f}%")

# """
# 총 예측 개수: 714
# 정확한 예측 개수: 498
# 예측 정확도: 69.75%
# """

총 예측 개수: 714
정확한 예측 개수: 498
예측 정확도: 69.75%
